In [3]:
%load_ext autoreload

from seven_segments import *
seven_segment(three)
seven_segment(six)
seven_segment(one)
print("test1")
seven_segment(test1)
print("test2")
seven_segment(test2)

# We will import the pylab namespace so that these answers
# are more readable to SEMT students more familiar with Matlab
from pylab import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 __ 
 __|
 __|
3
 __ 
|__ 
|__|
6
   
   |
   |
1
test1
 __ 
 __|
 __|
0
test2
 __ 
|__|
|__|
0


### Create the weight matrix

In [50]:
# Number of elements in each pattern
d = 11

# Patterns to store in the Hopfield network
X = array([six,three,one])
N = len(X)

# There are many ways to create a weight matrix. All are equally valid

# Example 1:
# Using np.sum, np.outer, and np.fill_diagonal
W = mean([outer(x,x) for x in X],axis=0)
fill_diagonal(W,0)

# Example 2: 
# using np.sum
# calculating outer product by reshaping to column * row vector
W2 = sum([x[:,None]@x[None,:] for x in X],axis=0)/N
W2 = W2 - diag(diag(W2)) # zeroing out with subtraction
assert allclose(W,W2)

# Example 3: 
# As a loop over patterns, 
# updating Wij as in Equation (3) in the PDF
Wloop = zeros((d,d))
for x in X:
    for i in range(d):
        for j in range(d):
            if i!=j: # Leave the diagonal zero
                Wloop[i,j] += x[i]*x[j]
Wloop /= N
assert allclose(W,Wloop)

# Example 4: 
# All matrix math, one line
Wmatrix = (X.T@X/N)*(1-eye(d))
assert allclose(W,Wmatrix)

## Evolving the Hopfield network and printing the energy

I organized this answer into a function, but this is optional. 

In [51]:
# Nonlinearity and energy

# Hopfield nonlinearity mapping to {-1,1}
sgn = lambda x:(np.array(x)>0)*2-1

# Hopfield energy
E   = lambda x,W:-0.5*x.T@W@x

# Function to iterate network states
# and, optionally, print
def converge(s,W,maxiter=20,printend=False,printenergy=False):
    if printenergy: 
        print(E(s,W))
    ps = None # detect 2-cycles
    for i in range(maxiter):
        s_ = sgn(W@s)
        if np.all(s==s_): break
        if ps is not None and np.all(ps==s_): 
            print("cycle detected")
            break
        s,ps = s_,s
        if printenergy: print(E(s,W))
    if printend:
        seven_segment(s)
        print(E(s,W))
        print()
    return s


# Function to iterate network states
# and, optionally, print
def converge(s,W,maxiter=20,printend=False,printenergy=False):
    if printenergy: 
        print(E(s,W))
    previous_energy = E(s,W)
    for i in range(maxiter):
        s = sgn(W@s)
        energy = E(s,W)
        if printenergy: 
            print('iteration %2d: E = %6.2f'%(i,energy))
        if energy>=previous_energy: break
        previous_energy = energy
    if printend:
        seven_segment(s)
        print(previous_energy)
        print()
    return s

# Test pattern 1 error corrects to 3
print("test1")
seven_segment(test1)
converge(test1,W,printend=True,printenergy=True)

# Test pattern 2 error corrects to 6
print("test2")
seven_segment(test2)
converge(test2,W,printend=True,printenergy=True)

test1
 __ 
 __|
 __|
0
-4.333333333333333
iteration  0: E = -16.33
iteration  1: E = -16.33
 __ 
 __|
 __|
3
-16.333333333333332

test2
 __ 
|__|
|__|
0
-0.3333333333333337
iteration  0: E =  -7.00
iteration  1: E =  -8.33
iteration  2: E = -23.00
iteration  3: E = -23.00
 __ 
|__ 
|__|
6
-23.0



array([ 1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1])

## Storing too much (optional)

In [52]:
W = mean([outer(a,a) for a in hexdigits],axis=0)
fill_diagonal(W,0)

for a,s in enumerate(hexdigits):
    for i in range(20):
        s_ = sgn(W@s)
        if np.all(s==s_): break
        s = s_
    print("\nThis pattern should be 0x%X:"%a)
    seven_segment(converge(s,W))


This pattern should be 0x0:
 __ 
|__|
|__|
8

This pattern should be 0x1:
   
   |
   |
1

This pattern should be 0x2:
 __ 
|__ 
|__ 
E

This pattern should be 0x3:
   
   |
   |
1

This pattern should be 0x4:
   
   |
   |
1

This pattern should be 0x5:
 __ 
|__ 
|__ 
E

This pattern should be 0x6:
 __ 
|__ 
|__ 
E

This pattern should be 0x7:
   
   |
   |
1

This pattern should be 0x8:
 __ 
|__|
|__|
8

This pattern should be 0x9:
 __ 
|__|
|__|
8

This pattern should be 0xA:
 __ 
|__ 
|__ 
E

This pattern should be 0xB:
 __ 
|__ 
|__ 
E

This pattern should be 0xC:
 __ 
|__ 
|__ 
E

This pattern should be 0xD:
   
   |
   |
1

This pattern should be 0xE:
 __ 
|__ 
|__ 
E

This pattern should be 0xF:
 __ 
|__ 
|__ 
E
